# Classification rules algorithm

## 1. Designing the PRISM algorithm

First, we need datastructures to store the rules. The Rule consists of antecedent (Left Hand Side) and consequent (Right Hand Side).  The LHS includes multiple conditions joined with and, and RHS is a class label. The Rule also needs to store its accuracy and coverage.

The list of conditions contains several objects of class Condition. Each condition includes the column name and the value. If the value is numeric, then the condition also includes an additional field `true_false` and is expressed in form: *if col >= val then True* or *if col >= val then False*.

In [1]:
class Rule:
    def __init__(self, class_label):
        self.conditions = []  # list of conditions
        self.class_label = class_label  # rule class

    def add_condition(self, condition):
        self.conditions.append(condition)

    def set_params(self, accuracy, coverage):
        self.accuracy = accuracy
        self.coverage = coverage

    def __repr__(self):
        return "If {} then {}. Coverage:{}, accuracy: {}".format(self.conditions, self.class_label,
                                                                 self.coverage, self.accuracy)


class Condition:
    def __init__(self, attribute, value, true_false = None):
        self.attribute = attribute
        self.value = value
        self.true_false = true_false

    def __repr__(self):
        if self.true_false is None:
            return "{}={}".format(self.attribute, self.value)
        else:
            return "{}>={}:{}".format(self.attribute, self.value, self.true_false)

Next comes the `learn_one_rule` algorithm. The required parameters are the names of the columns, the current subset of data, and the class label. The optional parameters are thresholds `min_coverage` and `min_accuracy`. Inaddition, sometimes we pass an already existing Rule in order to learn more refined condition. Now, if the Rule can be improved, we will return the new rule and the subset of data covered by the Rule. If we could not improve it - we return the original Rule and the original covered subset (`prev_covered`).

In [2]:
def learn_one_rule(columns, data, class_label, 
                   prev_rule=None, min_coverage=30,
                   min_accuracy=0.6, prev_covered=None):
    current_subset = data.copy()

    current_rule = prev_rule
    current_accuracy = 0
    current_coverage = None
    covered_subset = None
    if current_rule is not None:
        current_accuracy = current_rule.accuracy
        current_coverage = current_rule.coverage
        covered_subset = prev_covered.copy()
    best_col = None
    best_val = None
    true_false = None

    for col in columns[:-1]:
        # Extract unique values from the column
        unique_vals = current_subset[col].unique().tolist()
        
        # Consider each unique value in turn
        for val in unique_vals:
            
            # The treatment is different for numeric and categorical attributes
            if isinstance(val, int) or isinstance(val, float):
                # Here we construct 2 conditions: 
                # if actual value >= val and if actual value < val
                acc = [None]*2
                cov = [None]*2
                total_get = len(current_subset[current_subset[col] >= val])
                correct_get = len(current_subset.loc[(current_subset[col] >= val)
                                                 & (current_subset[columns[-1]] == class_label),])
                total_lt = len(current_subset) - total_get
                correct_lt = len(current_subset.loc[(current_subset[columns[-1]] == class_label),]) \
                             - correct_get
                if total_get >= min_coverage:
                    acc[0] = correct_get / total_get
                    cov[0] = total_get
                if total_lt >= min_coverage:
                    acc[1] = correct_lt/total_lt
                    cov[1] = total_lt

                # we select the best out of the 2
                best_i = None
                if acc[0] is not None and (acc[1] is None or acc[0] > acc[1]):
                    best_i = 0
                elif acc[1] is not None and (acc[0] is None or acc[1] > acc[0]):
                    best_i = 1
                elif acc[0] is not None and acc[1] is not None and acc[0] == acc[1]:
                    if total_get > total_lt:
                        best_i = 0
                    else:
                        best_i = 1
                
                # Now we see if the best of the 2 is better than the previous accuracy
                if best_i is not None:
                    if acc[best_i] > current_accuracy or \
                            (acc[best_i] == current_accuracy and cov[best_i] > current_coverage):
                        current_accuracy = acc[best_i]
                        current_coverage = cov[best_i]

                        best_col = col
                        best_val = val
                        true_false = bool(1 - best_i)
            else:
                # For categorical attributes - this is just single condition if actual value == val
                total = len(current_subset[current_subset[col] == val])
                if total < min_coverage:
                    continue
                correct = len(current_subset.loc[(current_subset[col] == val)
                                                 & (current_subset[columns[-1]] == class_label),])

                accuracy = correct / total

                if accuracy > current_accuracy:
                    current_accuracy = accuracy
                    current_coverage = total
                    best_col = col
                    best_val = val
                    true_false = None
                elif accuracy == current_accuracy:
                    if current_coverage is None or total > current_coverage:
                        current_accuracy = accuracy
                        current_coverage = total
                        best_col = col
                        best_val = val
                        true_false = None

            # print(best_col, best_val, current_accuracy, current_coverage)

    # If we managed to improve the rule accuracy
    if best_col is not None:
        # If the rule does not exist - create it
        if current_rule is None:
            current_rule = Rule(class_label)
        # Create a condition based on best_col, best_val and true_false
        condition = Condition(best_col, best_val, true_false)
        current_rule.add_condition(condition)
        current_rule.set_params(current_accuracy, current_coverage)
        
        # Generate a subset covered by this rule
        if true_false is None:
            covered_subset = current_subset[current_subset[best_col] == best_val]
        else:
            if true_false == True:
                covered_subset = current_subset[current_subset[best_col] >= best_val]
            else:
                covered_subset = current_subset[current_subset[best_col] < best_val]
    
    # If we did not refine the rule - return original rule and the original covered set
    return (current_rule, covered_subset, best_col)

The main algorithm `learn_rules` takes as parameters list of columns, with the last column representing the class label, and the original data in form of pandas dataframe. Optionally, you can pass the list of classes in order that you are interested to explore first. Two optional threshold parameters `min_coverage` and `min_accuracy` set up the conditions of rule's validity  for a specific dataset.

In [3]:
import pandas as pd
import numpy as np

def learn_rules (columns, data, classes=None, 
                 min_coverage = 30, min_accuracy = 0.6):
    # List of final rules
    rules = []
    
    # If list of classes of interest is not provided - it is extracted from the last column of data
    if classes is not None:
        class_labels = classes
    else:
        class_labels = data[columns[-1]].unique().tolist()

    current_data = data.copy()
    
    # This follows the logic of the original PRISm algorithm
    # It processes each class in turn. Because for high accuracy 
    # the rules generated are disjoint with respect to class label
    # this is not a problem when we are just interested in rules themselves - not classification
    # For classification the order in which the rules are discovered matters, and we should 
    # process all classes at the same time
    for class_label in class_labels:
        done = False
        while len(current_data) > min_coverage and not done:
            # Learn a rule with a single condition
            rule, covered, new_col = learn_one_rule(columns, current_data, class_label,
                                                    None, min_coverage, min_accuracy, None)
            # The best rule does not pass the coverage threshold - we are done with this class
            if rule is None:
                break

            # If we get the rule with coverage above threshold
            # We try to refine this rule
            if rule is not None:
                 # try to improve the rule
                while (new_col is not None and \
                       rule.accuracy < 1.0 and rule.coverage > min_coverage):   
                    # keep the previously covered in case we were unable to improve in this iteration
                    prev_covered = covered.copy()
                    
                    # remove the column already included in the rule condition
                    subset = covered.drop(columns=[new_col])
                    column_list = subset.columns.to_numpy().tolist()
                    rule, covered, new_col = learn_one_rule(column_list, subset, class_label,
                                                            rule, min_coverage, min_accuracy, 
                                                            prev_covered)

                # done with this rule
                if rule.accuracy >= min_accuracy:
                    rules.append(rule)

                    # remove rows covered by this rule
                    # you have to remove the rows where all of the conditions hold
                    # this is the only inefficient part of the implementation
                    # because I was forced to loop over data frame and look at all the involved columns
                    remaining_rows_index = []
                    for ind in current_data.index:
                        not_covered = False
                        conditions = rule.conditions
                        for cond in conditions:
                            if cond.true_false is None:
                                if current_data[cond.attribute][ind] != cond.value:
                                    not_covered = True
                                    break
                            elif cond.true_false == True:
                                if current_data[cond.attribute][ind] < cond.value:
                                    not_covered = True
                                    break
                            else:
                                if current_data[cond.attribute][ind] >= cond.value:
                                    not_covered = True
                                    break
                        if not_covered:
                            remaining_rows_index.append(ind)
                            
                    # These are the records after removing what was covered by the rule
                    current_data = current_data.loc[remaining_rows_index,]
                else:
                    done = True
            else:
                done = True
                
    return rules

## 2. The rules of survival. Titanic dataset

Titanic [dataset](https://drive.google.com/file/d/1yLR3Mieai6k4lFAPoMFD6AK7ImrJ1A93/view?usp=sharing).

In [4]:
data_file = "../../data_ml_2020/titanic.csv"

In [5]:
data = pd.read_csv(data_file)

data = data[['Pclass', 'Sex', 'Age', 'Survived']]

data = data.dropna(how="any")
print("Total rows", len(data))

column_list = data.columns.to_numpy().tolist()
print("Columns:", column_list)

Total rows 714
Columns: ['Pclass', 'Sex', 'Age', 'Survived']


In [6]:
# we can set different accuracy thresholds
# we can reorder class labels
rules = learn_rules(column_list, data, [1,0], 30, 0.7)
for rule in rules[:10]:
    print(rule)

If [Sex=female, Pclass>=2:False, Age>=26.0:True] then 1. Coverage:57, accuracy: 0.9824561403508771
If [Age>=6.0:False, Pclass>=2:True] then 1. Coverage:41, accuracy: 0.7073170731707317
If [Sex=female, Pclass>=3:False, Age>=24.0:False] then 1. Coverage:37, accuracy: 0.972972972972973
If [Sex=female, Pclass>=3:False, Age>=28.0:True] then 1. Coverage:41, accuracy: 0.926829268292683
If [Age>=54.0:True, Sex=male] then 0. Coverage:37, accuracy: 0.8918918918918919
If [Age>=39.0:True, Pclass>=2:True] then 0. Coverage:60, accuracy: 0.9166666666666666
If [Sex=male, Pclass>=2:True, Age>=32.5:True] then 0. Coverage:42, accuracy: 0.9761904761904762
If [Age>=24.0:False, Sex=male, Pclass>=2:True] then 0. Coverage:115, accuracy: 0.8782608695652174
If [Sex=male, Pclass>=2:True, Age>=27.0:False] then 0. Coverage:41, accuracy: 0.8780487804878049
If [Age>=28.0:True, Pclass>=2:True, Sex=male] then 0. Coverage:62, accuracy: 0.8064516129032258


## 3. Coronavirus symptoms and outcome

Coronavirus [dataset](https://drive.google.com/file/d/1auN6eSuHtWPXopcD7_bJhkfKYllZCk9Z/view?usp=sharing) (preprocessed as outlined [here](https://github.com/mgbarsky/ml_decision_tree_demo/blob/master/decision_tree_algorithm.ipynb)).

In [7]:
data_file = "../../data_ml_2020/covid_categorical_good.csv"

In [8]:
data = pd.read_csv(data_file)
data = data.dropna(how="any")
data.columns

Index(['sex', 'age', 'diabetes', 'copd', 'asthma', 'imm_supr', 'hypertension',
       'cardiovascular', 'obesity', 'renal_chronic', 'tobacco', 'outcome'],
      dtype='object')

Let's first try with categorical data only - by removing the only numerical column *age*.

In [9]:
data_categorical = data.drop(columns=['age'])
column_list = data_categorical.columns.to_numpy().tolist()
column_list

['sex',
 'diabetes',
 'copd',
 'asthma',
 'imm_supr',
 'hypertension',
 'cardiovascular',
 'obesity',
 'renal_chronic',
 'tobacco',
 'outcome']

In [10]:
# I really want to know what makes covid deadly
class_labels = ["dead"]
rules = learn_rules (column_list, data_categorical, class_labels,30, 0.6)
for rule in rules[:20]:
    print(rule)

If [renal_chronic=yes, diabetes=yes, cardiovascular=yes, obesity=no, sex=male, imm_supr=no, hypertension=yes, asthma=no] then dead. Coverage:70, accuracy: 0.6571428571428571
If [renal_chronic=yes, diabetes=yes, obesity=no, copd=yes, tobacco=no, hypertension=yes, imm_supr=no, asthma=no, sex=female] then dead. Coverage:31, accuracy: 0.6129032258064516


Now let's try on both classes and for the entire dataset including age.

In [11]:
column_list = data.columns.to_numpy().tolist()
column_list

['sex',
 'age',
 'diabetes',
 'copd',
 'asthma',
 'imm_supr',
 'hypertension',
 'cardiovascular',
 'obesity',
 'renal_chronic',
 'tobacco',
 'outcome']

In [12]:
# This runs for 12 minutes
rules = learn_rules (column_list, data, None, 30, 0.9)
for rule in rules[:20]:
    print(rule)

If [age>=26:False, tobacco=yes, asthma=yes] then alive. Coverage:47, accuracy: 1.0
If [age>=26:False, tobacco=yes, sex=female, obesity=yes] then alive. Coverage:83, accuracy: 1.0
If [age>=26:False, tobacco=yes, obesity=no, hypertension=no, sex=female] then alive. Coverage:274, accuracy: 0.9963503649635036
If [age>=26:False, hypertension=no, tobacco=yes, obesity=no, renal_chronic=no, imm_supr=no] then alive. Coverage:683, accuracy: 0.9970717423133236
If [age>=29:False, hypertension=no, sex=female, tobacco=yes, imm_supr=no] then alive. Coverage:331, accuracy: 1.0
If [age>=26:False, asthma=yes, obesity=no, sex=female] then alive. Coverage:246, accuracy: 1.0
If [age>=26:False, hypertension=no, sex=female, imm_supr=no, obesity=no, diabetes=no, renal_chronic=no, cardiovascular=no] then alive. Coverage:7773, accuracy: 0.9949826321883443
If [age>=30:False, hypertension=no, obesity=no, sex=female, imm_supr=no, tobacco=yes] then alive. Coverage:96, accuracy: 1.0
If [age>=30:False, hypertension=n

### 3. Final test. Contact lenses

Finally, let's test the algorithm on the original dataset from the PRISM paper - to see what rules does our algorithm produce.

The dataset can be downloaded from [here](https://docs.google.com/spreadsheets/d/1z_o28etWJxDdiLV5An_Ni34CZGEWrQGLmngHE79EQmM/edit?usp=sharing). The details of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Lenses).

**Attribute Information**:
3 Classes:
- 1 : the patient should be fitted with hard contact lenses,
- 2 : the patient should be fitted with soft contact lenses,
- 3 : the patient should not be fitted with contact lenses.

Features:
1. age of the patient: (1) young, (2) pre-presbyopic, (3) presbyopic
2. spectacle prescription:  (1) myope, (2) hypermetrope
3. astigmatic:     (1) no, (2) yes
4. tear production rate:  (1) reduced, (2) normal

Presbyopia is physiological insufficiency of accommodation associated with the aging of the eye that results in progressively worsening ability to focus clearly on close objects. So presbiopic means old.

Hypermetropia: far-sightedness, also known as long-sightedness - cannot see close.
Myopia: nearsightedness - cannot see at distance.

In [13]:
data_file = "../../data_ml_2020/contact_lenses.csv"

In [14]:
data = pd.read_csv(data_file, index_col=['id'])
data.columns

Index(['age', 'spectacles', 'astigmatism', 'tear production rate',
       'lenses type'],
      dtype='object')

In [15]:
len(data)

24

Replace numbers with actual values - for clarity.

In [16]:
# classes
conditions = [ data['lenses type'].eq(1), data['lenses type'].eq(2), data['lenses type'].eq(3)]
choices = ["hard","soft","none"]

data['lenses type'] = np.select(conditions, choices)

# age groups
conditions = [ data['age'].eq(1), data['age'].eq(2), data['age'].eq(3)]
choices = ["young","medium","old"]

data['age'] = np.select(conditions, choices)

# spectacles
conditions = [ data['spectacles'].eq(1), data['spectacles'].eq(2)]
choices = ["nearsighted","farsighted"]

data['spectacles'] = np.select(conditions, choices)

# astigmatism
conditions = [ data['astigmatism'].eq(1), data['astigmatism'].eq(2)]
choices = ["no","yes"]

data['astigmatism'] = np.select(conditions, choices)

# tear production rate
conditions = [ data['tear production rate'].eq(1), data['tear production rate'].eq(2)]
choices = ["reduced","normal"]

data['tear production rate'] = np.select(conditions, choices)

In [17]:
column_list = data.columns.to_numpy().tolist()
column_list

['age', 'spectacles', 'astigmatism', 'tear production rate', 'lenses type']

In [18]:
rules = learn_rules (column_list, data, None, 1, 0.95)
for rule in rules[:20]:
    print(rule)

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0
If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1.0
If [astigmatism=no, age=young] then soft. Coverage:1, accuracy: 1.0
If [astigmatism=no, age=medium] then soft. Coverage:1, accuracy: 1.0
If [age=young] then hard. Coverage:2, accuracy: 1.0
If [spectacles=nearsighted, astigmatism=yes] then hard. Coverage:2, accuracy: 1.0


Copyright &copy; 2020 Marina Barsky. All rights reserved.